Based on the material in https://github.com/random-forests/tutorials/blob/master/decision_tree.ipynb

### Import needed libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
%matplotlib inline

In [5]:
data_raw = pd.read_csv("spanish_power_market.csv").iloc[:8760, :]
data = data_raw[["datetime", "demand", "wind", "solar", "imbalance"]]

### Data preprocessing

In [6]:
data["datetime"] = pd.to_datetime(data["datetime"], utc=True)

# basically a boolean specifying whether or not there is need for positive energy
data["target"] = np.where(data["imbalance"]>=0, "positive", "negative")

data.drop(["datetime", "imbalance"], axis=1, inplace=True)
data.head()

,demand,wind,solar,target
0,23459.000000,3794.191,15.986,positive
1,22781.000000,3767.804,15.921,positive
2,21448.500000,3757.543,8.272,negative
3,20262.166667,3716.763,0.139,negative
4,19463.500000,3532.569,0.233,negative


### Splitting data into training and target: x and y

In [7]:
x = data.drop("target", axis=1)
y = data[["target"]]

### Creating a partition

In [8]:
def partition(
    data: pd.DataFrame, 
    feature: str, 
    split_value: float
): 
    """
    Creates two partitions from a slice of a dataframe according to a 
    certain split value
        
    :param data: table to be split (type: pd.DataFrame)
    :param feature: variable/column on which we will split the data (type: str)
    :param split_value: value that will cut the data into two partitions 
        (left: positive energy, right: negative energy) (type: float)
    :return (left, right): tuple containing the indices of the observations in each partition (type: tuple)
    """
    left_partition_index = data[data[feature] <= split_value].index
    right_partition_index = data[data[feature] > split_value].index

    return left_partition_index, right_partition_index

In [9]:
# test for first node: splitting x on wind at 5000
left, right = partition(x, "wind", 5000.0)
left, right

(Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
             ...
             8750, 8751, 8752, 8753, 8754, 8755, 8756, 8757, 8758, 8759],
            dtype='int64', length=4035),
 Int64Index([  21,   22,   23,   24,   25,   26,   27,   28,   29,   30,
             ...
             8617, 8618, 8619, 8620, 8621, 8623, 8624, 8625, 8626, 8627],
            dtype='int64', length=4725))

### Impurity: how mixed is our target slice

In [10]:
def label_count(target):
    """
    Returns unique labels and count per label
    
    :param target: target column of the dataset (type: pd.Series)
    :return dict_label_count: dictionary containing the unique classes and their counts (type: dict)
    """
    unique_labels_in_partition, unique_label_counts = np.unique(target, return_counts=True)
    dict_label_count = dict(zip(unique_labels_in_partition, unique_label_counts))
    return dict_label_count

In [11]:
def gini_impurity(target):
    """
    How mixed is the label: 
    p_i = proportion of missclassified items
    H(Gini) = Sum_i{p_i * (1 - p_i)}
    
    :param target: target column of the dataset (type: pd.Series)
    :return gini_imp: gini impurity value (type: float)
    """
    # the unique labels and counts in the data
    dict_label_count = label_count(target)
    total_labels = np.sum(list(dict_label_count.values()))
    
    # calculate impurity
    gini_imp = 0
    for unique_labels, unique_label_count in dict_label_count.items():
        p_i = unique_label_count / total_labels
        gini_imp += p_i * (1 - p_i)
        
    return gini_imp    

In [12]:
# test for first node: calculate the gini impurity of the first node
gini_y = gini_impurity(y)
gini_y

0.48285792101916136

### Information Gain: how much the impurity is improved by creating a split

In [13]:
def information_gain(target_at_node, impurity_at_node, left_partition_index, right_partition_index): 
    """
    For each node we calculate the information gain for each possible split. Information Gain
    means how much the impurity is reduced by splitting on a certain split value.
    
    :param target_at_node: slice of total target that made it into that node
    :param impurity_at_node: gini_impurity of target_at_node
    :param left_partition_index: left slice from partition
    :param right_partition_index: right slice from partition
    :return info_gain: how much the impurity is reduced by splitting
    """
    
    p = len(left_partition_index) / (len(left_partition_index) + len(right_partition_index))
    info_gain = (
        impurity_at_node 
        - p * gini_impurity(target_at_node.loc[left_partition_index]) 
        - (1 - p) * gini_impurity(target_at_node.loc[right_partition_index])
    )
    return info_gain

In [14]:
info_gain = information_gain(y, gini_y, left, right)
info_gain

0.0014345356038559243

### Finding the best features and split_values on which to split: best_split

In [15]:
def best_split(data, target):
    """
    Splits the data on the best column and value 
    
    :param data: the training data
    :param target: the target label 
    :return best_gain, best_col, best_val: (type: tuple)
        best_gain: value of Information Gain by splitting on best_feature at best_split_value
        best_feature: feature of data that deals the best partitions
        best_split: value of best_feature that deals the best partitions
    """
    features = data.columns
    best_gain = 0 
    best_feature = None
    best_split_value = None
    
    target_idx = target.index # getting the index of the labels
    impurity = gini_impurity(target) # determining the impurity at the current node
    
    # loop for each training feature
    for feature in features: 
        
        unique_values = set(data[feature])
        
        # loop for each unique value in our training feature
        for split_value in unique_values: 
            
            left_partition_index, right_partition_index = partition(data, feature, split_value)
                
            # calculate info_gain
            info_gain = information_gain(target, impurity, left_partition_index, right_partition_index)
            
            # select feature and split_value if improves current gain
            if info_gain > best_gain:
                best_gain, best_feature, best_split_value = info_gain, feature, split_value
                
    return best_gain, best_feature, best_split_value

In [16]:
best_split(x.iloc[:10, :], y.iloc[:10, :])

(0.32, 'demand', 21448.5)

### Sh*t gets real: classes

In [17]:
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, target):
        self.predictions = label_count(target)
        
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """ 
    def __init__(self,
                 feature,
                 split_value,
                 true_branch,
                 false_branch):
        self.feature = feature
        self.split_value = split_value
        self.true_branch = true_branch
        self.false_branch = false_branch

In [18]:
def print_tree(node, spacing=""):
    """
    World's most elegant tree printing function.
    
    Input
    node: the tree node
    spacing: used to space creating tree like structure
    """

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the feature and split_value at this node
    print(spacing + f"[{node.feature} <= {node.split_value}]")
    

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

### Recursive partitioning

In [19]:
def build_tree(data, target): 
    """
    Recursive partition implementation for building a tree
    
    :param df: the training data
    :param label: the target labels
    :return Tree: Decision tree (type: Decision_Node) 
    """
    # start splitting with the original data: root node
    best_gain, best_feature, best_split_value = best_split(data, target)
    
    # if perfectly classified then gain = 0
    if best_gain == 0: 
        return Leaf(target)
    
    left_partition_index, right_partition_index = partition(data, best_feature, best_split_value)
    
    true_observations = build_tree(data.loc[left_partition_index], target.loc[left_partition_index])
    
    false_observations = build_tree(data.loc[right_partition_index], target.loc[right_partition_index])
    
    return Decision_Node(best_feature, best_split_value, true_observations, false_observations)

In [20]:
tree = build_tree(x.loc[:500, :], y.loc[:500, :])

In [21]:
print_tree(tree)

[demand <= 21448.5]
--> True:
  Predict {'negative': 13}
--> False:
  [wind <= 1888.775]
  --> True:
    [demand <= 37007.0]
    --> True:
      Predict {'negative': 12}
    --> False:
      [solar <= 10.758]
      --> True:
        Predict {'negative': 3}
      --> False:
        Predict {'positive': 4}
  --> False:
    [wind <= 7539.071999999999]
    --> True:
      [demand <= 32067.33333333333]
      --> True:
        [solar <= 3717.063]
        --> True:
          [wind <= 2536.6910000000003]
          --> True:
            Predict {'negative': 3}
          --> False:
            [demand <= 31861.83333333333]
            --> True:
              [wind <= 5128.068]
              --> True:
                [wind <= 3176.691]
                --> True:
                  [solar <= 9.616]
                  --> True:
                    Predict {'positive': 13}
                  --> False:
                    [solar <= 9.753]
                    --> True:
                      Predict {'neg

### Using our trained tree to predict!

In [22]:
def predict(test_data, tree):
    
    """
    Function to infer data from a trained tree
    
    :param test_data: Unseen observation (type: pd.DataFrame)
    :param tree: tree that has been trained on training data (type: Decision_Node)
    :return prediction: predicted class to which it belongs according to the model (type: str)
    """
    
    # Check if we are at a leaf node
    if isinstance(tree, Leaf): 
        return max(tree.predictions)
    
    # the current feature_name and value 
    feature, split_value = tree.feature, tree.split_value
    
    # pass the observation through the nodes recursively
    if test_data[feature] <= split_value: 
        return predict(test_data, tree.true_branch)
    
    else: 
        return predict(test_data, tree.false_branch)

#### predicting a single value

In [23]:
x_test, y_test = x.iloc[1500], y.iloc[1500]
y_hat = predict(x_test, tree)

print(f"Actual value: {y_test.values[0]}")
print(f"Predicted value: {y_hat}")

Actual value: negative
Predicted value: positive


### predicting for a series

In [24]:
# create a new col of predictions
x_test, y_test = x.iloc[-100: , :], y.iloc[-100: , :]
y_hat = x_test.apply(lambda row: predict(row, tree), axis=1)

# using sklearn accuracy let's measure the performance of our model
acc = accuracy_score(y_test, y_hat)

In [25]:
print("OVERFITTING!")
print(f"Accuracy on test set: {acc:.2f}")

OVERFITTING!
Accuracy on test set: 0.27
